Plan:
make a tree with 15 + 2*2 = 19 nodes. 15 standard ones and 2 branches splitting off from studying

For each Node need:
node.name == move that leads to that node
node.player == player to make the decision from the node
node.children <-- taken care of by anytree
node.parent == pointer to parent node

artist == player 1

buyer == player 2

nature == player 3

In [1]:
from anytree import Node, RenderTree
import anytree
import copy
import random

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
import numpy as np


In [2]:
def create_tree(p):
    node0 = Node("create_art", player = 1)

    node1 = Node("high_price",parent = node0, player = 2)
    node2 = Node("low_price",parent = node0,  player = 2)

    node3 = Node("learn",parent = node1,  player = 3)
    node4 = Node("guess",parent = node1,  player = 2)
    node5 = Node("learn",parent = node2,  player = 3)
    node6 = Node("guess",parent = node2,  player = 2)

    node7 = Node("good_art",parent = node3,  player = 2, prob = p)
    node8 = Node("bad_art",parent = node3,  player = 2, prob = 1-p)
    node9 = Node("buy",parent = node4,  player = 2)
    node10 = Node("not_buy",parent = node4,  player = 2)
    node11 = Node("good_art",parent = node5,  player = 2, prob = p)
    node12 = Node("bad_art",parent = node5,  player = 2, prob = 1-p)
    node13 = Node("buy",parent = node6,  player = 2)
    node14 = Node("not_buy",parent = node6,  player = 2)

    node15 = Node("buy",parent = node7,  player = 2)
    node16 = Node("not_buy",parent = node7,  player = 2)
    node17 = Node("buy",parent = node8,  player = 2)
    node18 = Node("not_buy",parent = node8,  player = 2)
    node19 = Node("buy",parent = node11,  player = 2)
    node20 = Node("not_buy",parent = node11,  player = 2)
    node21 = Node("buy",parent = node12,  player = 2)
    node22 = Node("not_buy",parent = node12,  player = 2)

    return node0

def set_payoffs_for_leaf_nodes(root_node, high_price, low_price, cost_to_learn, good_art_value, bad_art_value, p):
    if high_price < low_price or low_price < 0 or cost_to_learn < 0 or good_art_value < bad_art_value \
    or p < 0 or p > 1:
        print('ERROR. SOME OF THE PARAMETERS DO NOT MAKE PHYSICAL SENSE')
    leaf_nodes = [node for node in anytree.iterators.preorderiter.PreOrderIter(root_node) if node.is_leaf]
    leaf_nodes[0].payoff = {1:high_price, 2: -high_price-cost_to_learn+good_art_value}
    leaf_nodes[1].payoff = {1:0, 2: -cost_to_learn}
    leaf_nodes[2].payoff = {1:high_price, 2: -high_price-cost_to_learn+bad_art_value}
    leaf_nodes[3].payoff = {1:0, 2: -cost_to_learn}
    leaf_nodes[4].payoff = {1:high_price, 2: -high_price + p*good_art_value +(1-p)*bad_art_value }
    leaf_nodes[5].payoff = {1:0, 2: 0}

    leaf_nodes[6].payoff = {1:low_price, 2: -low_price-cost_to_learn+good_art_value}
    leaf_nodes[7].payoff = {1:0, 2: -cost_to_learn}
    leaf_nodes[8].payoff = {1:low_price, 2: -low_price-cost_to_learn+bad_art_value}
    leaf_nodes[9].payoff = {1:0, 2: -cost_to_learn}
    leaf_nodes[10].payoff = {1:low_price, 2: -low_price + p*good_art_value +(1-p)*bad_art_value }
    leaf_nodes[11].payoff = {1:0, 2: 0}
    return



def exp(node):
    '''returns expectation of the node'''
    if node.is_leaf:
        return node.payoff
    if node.player != 3:
        return exp(max( node.children, key = lambda child_node: exp(child_node)[node.player] ))
    #expectation for node if player == 3, i.e. nature
    return {1:round(sum([child_node.prob*exp(child_node)[1] for child_node in node.children]),2), \
            2:round(sum([child_node.prob*exp(child_node)[2] for child_node in node.children]),2)}

def solve(node):
    if node.is_leaf:   #base case
        return node
    elif node.player == 3:
        #decide at random
        if random.random() < node.children[0].prob:
            return solve(node.children[0])
        else:
            return solve(node.children[1])
    return solve(max(node.children, key = lambda child_node: exp(child_node)[node.player]))

In [3]:
def set_probs(root_node, p):
    root_node.children[0].children[0].children[0].prob = p
    root_node.children[0].children[0].children[1].prob = 1-p
    root_node.children[1].children[0].children[0].prob = p
    root_node.children[1].children[0].children[1].prob = 1-p

In [4]:
def set_payoffs_and_solve(high_price, low_price, cost_to_learn, good_art_value, bad_art_value):
    set_payoffs_for_leaf_nodes(node0, high_price, low_price, cost_to_learn, good_art_value, bad_art_value, p)
    e = exp(node0)
    b = [node.name for node in solve(node0).path]
    
    print('expected payoffs: '+str(e))
    print("behavior at equilibrium: "+str(b))
    #result = 'expected payoffs: '+str(e) +"behavior: "+str(b) 
    return

In [5]:
######################## INTERACTIVE ################
high_price = widgets.FloatSlider(min=8,max=20,step=1,value=8)             #low price
low_price = widgets.FloatSlider(min=0,max=8,step=0.5,value=3)             #low price

good_art_value=widgets.FloatSlider(min=10,max=50,step=1,value=20)               #value of good art
bad_art_value=widgets.FloatSlider(min=0,max=10,step=1,value=1)               #value of bad art

cost_to_learn = widgets.FloatSlider(min=0,max=10,step=0.5,value=2)            #cost of learning

p = 0.4
node0 = create_tree(p)
 

interact(set_payoffs_and_solve, high_price=high_price, \
         low_price=low_price, cost_to_learn=cost_to_learn, good_art_value=good_art_value, \
         bad_art_value=bad_art_value)

interactive(children=(FloatSlider(value=8.0, description='high_price', max=20.0, min=8.0, step=1.0), FloatSlider(value=3.0, description='low_price', max=8.0, step=0.5), FloatSlider(value=2.0, description='cost_to_learn', max=10.0, step=0.5), FloatSlider(value=20.0, description='good_art_value', max=50.0, min=10.0, step=1.0), FloatSlider(value=1.0, description='bad_art_value', max=10.0, step=1.0), Output()), _dom_classes=('widget-interact',))

<function __main__.set_payoffs_and_solve>

PLAN:
define functions of p for each behavior. So 8 functions in total. Each takes in p and outputs expectation of each behavior
define function that takes in all parameters and outputs two subplots, each one with 4 expectations, one for each behavior.
do interact on all parameters except p.


In [9]:
high_price = widgets.FloatSlider(min=8,max=20,step=1,value=8)             #low price
low_price = widgets.FloatSlider(min=0,max=8,step=0.5,value=3)             #low price
good_art_value=widgets.FloatSlider(min=10,max=50,step=1,value=20)               #value of good art
bad_art_value=widgets.FloatSlider(min=0,max=10,step=1,value=1)               #value of bad art
cost_to_learn = widgets.FloatSlider(min=0,max=10,step=0.5,value=2)            #cost of learning
node0 = create_tree(0)
def main(high_price, low_price, cost_to_learn, good_art_value, bad_art_value):
    high_price_expectations = []
    low_price_expectations = []
    
    learn_expectations = []
    guess_and_buy_expectations = []
    guess_and_not_buy_expectations = []
    p_list = [0.01*p for p in range(101)]
    for p in p_list:
        set_probs(node0, p)
        set_payoffs_for_leaf_nodes(node0, high_price, low_price, cost_to_learn, good_art_value, bad_art_value, p)

        high_price_expectations.append(exp(node0.children[0])[1])
        low_price_expectations.append(exp(node0.children[1])[1])
        
        low_price_is_set = int(exp(node0.children[0])[1] < exp(node0.children[1])[1])
        learn_expectations.append(exp(node0.children[low_price_is_set].children[0])[2])
        guess_and_buy_expectations.append(exp(node0.children[low_price_is_set].children[1].children[0])[2])
        guess_and_not_buy_expectations.append(exp(node0.children[low_price_is_set].children[1].children[1])[2])

        if p == 0.4:
            e = exp(node0)
            b = [node.name for node in solve(node0).path]
            print("For p = 0.4:")
            print('expected payoffs: '+str(e))
            print("behavior at equilibrium: "+str(b))

    plt.figure(1)
    
    plt.subplot(211)
    plt.plot(p_list, high_price_expectations, label = "high price")
    plt.plot(p_list, low_price_expectations, label = "low price")
    plt.title("Player 1")
    #plt.xlabel("probability that art is good")
    plt.ylabel("expectation")
    plt.legend()

    plt.subplot(212)
    plt.plot(p_list, learn_expectations, label = "learn")
    plt.plot(p_list, guess_and_buy_expectations, label = "guess and buy")
    plt.plot(p_list, guess_and_not_buy_expectations, label = "guess and not buy")
    plt.title("Player 2")
    plt.xlabel("probability that art is good")
    plt.ylabel("expectation")
    plt.legend()
    
    plt.subplots_adjust(left=None, bottom=0, right=1.5, top=1.2,
                wspace=None, hspace=0.5)
    
    return plt    
interact(main, high_price=high_price, \
         low_price=low_price, cost_to_learn=cost_to_learn, good_art_value=good_art_value, \
         bad_art_value=bad_art_value)



interactive(children=(FloatSlider(value=8.0, description='high_price', max=20.0, min=8.0, step=1.0), FloatSlider(value=3.0, description='low_price', max=8.0, step=0.5), FloatSlider(value=2.0, description='cost_to_learn', max=10.0, step=0.5), FloatSlider(value=20.0, description='good_art_value', max=50.0, min=10.0, step=1.0), FloatSlider(value=1.0, description='bad_art_value', max=10.0, step=1.0), Output()), _dom_classes=('widget-interact',))

<function __main__.main>

################
TESTING:

In [10]:
node0 = create_tree(0.4)

In [11]:
###### TEST FOR set_payoffs_for_leaf_nodes() ######
###### OUTPUT: (8,5) (0,-2) (8, -9) (0,-2) (8,-1.4) (0,0) (3,10) (0,-2) (3, -4) (0,-2) (3,3.6) (0,0) ######
set_payoffs_for_leaf_nodes(node0, 8, 3, 2, 15, 1, 0.4)
[node for node in anytree.iterators.preorderiter.PreOrderIter(node0) if node.is_leaf]

[Node('/create_art/high_price/learn/good_art/buy', payoff={1: 8, 2: 5}, player=2),
 Node('/create_art/high_price/learn/good_art/not_buy', payoff={1: 0, 2: -2}, player=2),
 Node('/create_art/high_price/learn/bad_art/buy', payoff={1: 8, 2: -9}, player=2),
 Node('/create_art/high_price/learn/bad_art/not_buy', payoff={1: 0, 2: -2}, player=2),
 Node('/create_art/high_price/guess/buy', payoff={1: 8, 2: -1.4}, player=2),
 Node('/create_art/high_price/guess/not_buy', payoff={1: 0, 2: 0}, player=2),
 Node('/create_art/low_price/learn/good_art/buy', payoff={1: 3, 2: 10}, player=2),
 Node('/create_art/low_price/learn/good_art/not_buy', payoff={1: 0, 2: -2}, player=2),
 Node('/create_art/low_price/learn/bad_art/buy', payoff={1: 3, 2: -4}, player=2),
 Node('/create_art/low_price/learn/bad_art/not_buy', payoff={1: 0, 2: -2}, player=2),
 Node('/create_art/low_price/guess/buy', payoff={1: 3, 2: 3.6}, player=2),
 Node('/create_art/low_price/guess/not_buy', payoff={1: 0, 2: 0}, player=2)]

In [ ]:
###### TEST FOR exp() ########
###### OUTPUT: {1: 3.2, 2: 0.8} #########
exp(node0.children[0].children[0])

In [ ]:
###### TEST FOR solve() ########
###### OUTPUT: either {1: 8, 2: 5} or {1: 0, 2: -2} #########
solve(node0.children[0].children[0])

In [ ]:
print(RenderTree(node0))

END OF TESTING
###################